In [6]:
#importing all libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torchvision
from scipy.io import loadmat
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, confusion_matrix
from scipy.io import loadmat
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

In [7]:
# Load the MNIST dataset
mnist = loadmat("C:\\Users\\rohan\\OneDrive - Indian Institute of Technology Bombay\\IITB\\WiDS\\mnist-original.mat")

# Extract data and labels
data = mnist['data']
labels = mnist['label'][0]

# Convert to PyTorch tensors
data_tensor = torch.Tensor(data).T
labels_tensor = torch.Tensor(labels)

# Creating a reshaped tensor which has the same dimensions as that of the original image
reshaped_data = data_tensor.view(70000, 28, 28)

In [8]:
# Splitting into training and testing data
train_data , test_data = train_test_split(reshaped_data, test_size = 0.2)

In [9]:
numb_batch = 64

In [10]:
T = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])
train_data = torchvision.datasets.MNIST('mnist_data', train=True, download=True, transform=T)
val_data = torchvision.datasets.MNIST('mnist_data', train=False, download=True, transform=T)

train_dl = torch.utils.data.DataLoader(train_data, batch_size = numb_batch)
val_dl = torch.utils.data.DataLoader(val_data, batch_size = numb_batch)

100.0%


Extracting mnist_data\MNIST\raw\train-images-idx3-ubyte.gz to mnist_data\MNIST\raw



100.0%


Extracting mnist_data\MNIST\raw\train-labels-idx1-ubyte.gz to mnist_data\MNIST\raw



100.0%


Extracting mnist_data\MNIST\raw\t10k-images-idx3-ubyte.gz to mnist_data\MNIST\raw



100.0%

Extracting mnist_data\MNIST\raw\t10k-labels-idx1-ubyte.gz to mnist_data\MNIST\raw



In [11]:
def create_lenet():
    model = nn.Sequential(
        nn.Conv2d(1, 6, 5, padding=2),
        nn.ReLU(),
        nn.AvgPool2d(2, stride=2),
        nn.Conv2d(6, 16, 5, padding=0),
        nn.ReLU(),
        nn.AvgPool2d(2, stride=2),
        nn.Flatten(),
        nn.Linear(400, 120),
        nn.ReLU(),
        nn.Linear(120, 84),
        nn.ReLU(),
        nn.Linear(84, 10)
    )
    return model

In [12]:
def validate(model, data):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(data):
        images = images.cuda()
        x = model(images)
        value, pred = torch.max(x,1)
        pred = pred.data.cpu()
        total += x.size(0)
        correct += torch.sum(pred == labels)
    return correct*100./total

In [13]:
def train(model, train_dl, val_dl, numb_epoch=3, lr=1e-3):
    accuracies = []
    cec = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    max_accuracy = 0
    for epoch in range(numb_epoch):
        for i, (images, labels) in enumerate(train_dl):
            optimizer.zero_grad()
            pred = model(images)
            loss = cec(pred, labels)
            loss.backward()
            optimizer.step()
        accuracy = float(validate(model, val_dl))
        accuracies.append(accuracy)
        if accuracy > max_accuracy:
            best_model = copy.deepcopy(model)
            max_accuracy = accuracy
            print("Best Model Accuracy: ", accuracy)
        print('Epoch:', epoch+1, "Accuracy:", accuracy, '%')
    plt.plot(accuracies)
    return best_model

In [8]:
# Convert the NumPy arrays to PyTorch tensors
train_data_tensor = torch.Tensor(train_data)
test_data_tensor = torch.Tensor(test_data)

# Reshape the data to match the original image dimensions
train_data_reshaped = train_data_tensor.view(-1, 1, 28, 28)
test_data_reshaped = test_data_tensor.view(-1, 1, 28, 28)

# Convert labels to integers
labels_tensor = labels_tensor.long()

# Splitting into training and testing labels
train_labels, test_labels = train_test_split(labels_tensor.numpy(), test_size=0.2)
train_labels_tensor = torch.Tensor(train_labels).long()
test_labels_tensor = torch.Tensor(test_labels).long()

# Define the CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(-1, 64 * 7 * 7)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Instantiate the model, define loss function and optimizer
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Convert data and labels to PyTorch DataLoader
train_dataset = TensorDataset(train_data_reshaped, train_labels_tensor)
test_dataset = TensorDataset(test_data_reshaped, test_labels_tensor)

batch_size = 64
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Training the model
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    for batch_data, batch_labels in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_data)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()

    # Testing the model
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for batch_data, batch_labels in test_loader:
            outputs = model(batch_data)
            _, predicted = torch.max(outputs, 1)
            total += batch_labels.size(0)
            correct += (predicted == batch_labels).sum().item()

        accuracy = correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Test Accuracy: {accuracy:.4f}')

# Evaluate the final model on the entire test set
model.eval()
with torch.no_grad():
    test_outputs = model(test_data_reshaped)
    _, predicted_labels = torch.max(test_outputs, 1)

# Calculate and print the confusion matrix
conf_matrix = confusion_matrix(test_labels_tensor, predicted_labels)
print("Confusion Matrix:")
print(conf_matrix)

# Calculate and print the overall accuracy
accuracy = accuracy_score(test_labels_tensor, predicted_labels)
print(f"Overall Accuracy: {accuracy:.4f}")

# Save the trained model
torch.save(model.state_dict(), 'mnist_cnn_model.pth')


Epoch [1/10], Test Accuracy: 0.1144
Epoch [2/10], Test Accuracy: 0.1179
Epoch [3/10], Test Accuracy: 0.1171
Epoch [4/10], Test Accuracy: 0.1176
Epoch [5/10], Test Accuracy: 0.1139
Epoch [6/10], Test Accuracy: 0.1170
Epoch [7/10], Test Accuracy: 0.1174
Epoch [8/10], Test Accuracy: 0.1180
Epoch [9/10], Test Accuracy: 0.1172
Epoch [10/10], Test Accuracy: 0.1178
Confusion Matrix:
[[   0 1367    7    6    5    0   10   10    9    9]
 [   4 1570    9   12    4    0    8   17    5   16]
 [   3 1334    8    7    4    3    9   17   11   16]
 [   1 1362    6   19    0    0    2   18    9   16]
 [   3 1221    9   15    5    2    6   22    7    9]
 [   0 1222    3   15    6    0    1   22    8    7]
 [   3 1276    5   13    4    2    6   17    9    8]
 [   5 1359    9   12    3    1    5   25   14   14]
 [   1 1280    5    9    3    1    5   16    5   12]
 [   7 1299    7   17    5    3    4   16    8   11]]
Overall Accuracy: 0.1178


In [6]:
train_data_tensor.shape

torch.Size([627, 70000])